In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pandas import ExcelWriter

import math
import seaborn as sns
from matplotlib.pyplot import figure
from sklearn.metrics import f1_score, accuracy_score
import os


from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot

from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
import statsmodels.api as sm

# !pip install tsfresh --user
import tsfresh
from tsfresh import extract_relevant_features
from tsfresh import extract_features

os.chdir("C:/Users/VIP13/Статья 2/Котировки 6")

In [ ]:
# accuracy for the predictive (prog) and original(orig) data series
def accuracy(orig, prog):

    orig_1=orig[:-1]
    orig_2=[]
    for i in range(1, len(orig)):
        orig_2.append(orig[i])


    prog_1=prog[:-1]
    prog_2=[]
    for i in range(1, len(prog)):
        prog_2.append(prog[i])

    differ_fr=pd.DataFrame()
    differ_fr['orig_bin']=orig_2-orig_1
    differ_fr['prog_bin']=prog_2-prog_1

    differ_fr[differ_fr < 0] = 0
    differ_fr[differ_fr > 0] = 1
    accur=accuracy_score(differ_fr['orig_bin'], differ_fr['prog_bin'])
#     print(accur)
    return accur

# download data

In [ ]:
frame_3=pd.read_csv('all_snp_btc_rv_rr.csv', delimiter=';')
frame=frame_3.iloc[:,1:].copy() #на входе подаются квадраты волатильностей, поэтому сначала из них берем корни, а потом логарифмируем
frame=frame.loc[:, [c for c in frame.columns if (frame[c]>0).all()]]
frame=np.log(np.sqrt(frame))
ind=[i for i in range(0,len(frame))]
frame.index=ind
index_list=np.array(frame['rv_shk_btc'])
# index_list=np.array(frame['rv_shk_snp'])

# Just ARMA without exog 
The evaluation is done in a sliding window of length 399 values in increments of 5 values

In [ ]:
name=[]
for ar in range(1,10):
    for ma in range(0,9):
        name.append('arima'+'_'+str(ar)+'_'+str(ma))
frame_orig=pd.DataFrame(columns=name)
frame_pogr=pd.DataFrame(columns=name)

mape=[]
accur=[]

for ar in range(1,10):
    for ma in range(0,9):
        
        orig=[]
        prog=[]
        len_wind=399
        step_smooth=5
        iteratation_5=int((len(index_list)-len_wind)/step_smooth)+1
        i=0
        # iteratation_5=5
        for i in range(0,iteratation_5):
            train=index_list[0+step_smooth*i:len_wind+step_smooth*i]
        #     order_A2 = sm.tsa.arma_order_select_ic(train,ic='aic')['aic_min_order']   # AIC准则求解模型阶数p,q
        #     print (order_A2)   #各个模型阶次
            model_A2 =  ARMA(train,order=(ar,0,ma))   # 建立模型
            results_A2 = model_A2.fit()

            forec=index_list[0+1+step_smooth*i:len_wind+1+step_smooth*i]

            res=results_A2.forecast()[0][0]
            prr.append(model_A2.predict(params=results_A2.params,start=len(train)-2,end=len(train))[-1])

            orig.append(forec[-1])
            prog.append(res)
#             mape.append(abs((forec[-1]-res)/forec[-1]))
        orig=np.exp(orig)
        prog=np.exp(prog)
        frame_orig['arima'+'_'+str(ar)+'_'+str(ma)]=orig
        frame_pogr['arima'+'_'+str(ar)+'_'+str(ma)]=prog
        
        mape.append((abs((np.array(orig)-np.array(prog))/np.array(orig))).mean())
        accur.append((accuracy(orig, prog)))

In [ ]:
abs((frame_orig-frame_pogr)/frame_orig).mean().sort_values()
accur
mape

In [ ]:
# unload ready-made dataframes
frame_orig.to_csv('arima_btc_orig.csv')
frame_pogr.to_csv('arima_btc_prog.csv')

# frame_orig.to_csv('arima_snp_orig.csv')
# frame_pogr.to_csv('arima_snp_prog.csv')



# SARIMAX with exog regressors - features from tsfresh

In [ ]:
# extract relevant features for each interval for Bitcoin

data=frame['rv_shk_btc']
data=frame['rv_shk_snp']

n_in=5
len_wind=399
step_smooth=5

iteratation_5=int((len(index_list)-len_wind)/step_smooth)+1

train_exog=[]
for i in range(0,iteratation_5):
    datas=data.iloc[0+step_smooth*i:len_wind+step_smooth*i]
    X=pd.DataFrame(datas.iloc[:-1])
    y=datas.iloc[1:]
    y.index=np.arange(0,len(y),1)
    X.index=y.index
    X['id']=y.index
    features_filtered_direct = extract_relevant_features(X,y, column_id='id')


    X_a=pd.DataFrame(datas)
    X_a.index=np.arange(0,len(X_a),1)
    X_a['id']=X_a.index

    alls_feat=extract_features(X_a,column_id='id')

    train_exog1=alls_feat[features_filtered_direct.iloc[0:,1:].columns]
    train_exog.append(train_exog1)
    print(i)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

name=[]
for ar in range(1,10):
    name.append('arima'+'_'+str(ar))
frame_orig=pd.DataFrame(columns=name)
frame_pogr=pd.DataFrame(columns=name)

mape=[]
accur=[]

for ar in range(1,10):
        orig=[]
        prog=[]
        len_wind=399
        step_smooth=5
        iteratation_5=int((len(index_list)-len_wind)/step_smooth)+1
        i=0
        # iteratation_5=5
        for i in range(0,iteratation_5):
            train=index_list[0+step_smooth*i:len_wind+step_smooth*i]
        
        
            model_A2 =  sm.tsa.statespace.SARIMAX(train, order=(ar, 0,0), exog = train_exog[i],
                                                  trend='c'
                                                 ).fit()            

            results_A2 = model_A2

            forec=index_list[0+1+step_smooth*i:len_wind+1+step_smooth*i]


            res=results_A2.forecast(exog = train_exog[i][-1:]).iloc[0]

            orig.append(forec[-1])
            prog.append(res)
            
        orig=np.exp(orig)
        prog=np.exp(prog)
        frame_orig['arima'+'_'+str(ar)]=orig
        frame_pogr['arima'+'_'+str(ar)]=prog
        
        mape.append((abs((np.array(orig)-np.array(prog))/np.array(orig))).mean())
        accur.append((accuracy(orig, prog)))
        print('arima'+'_'+str(ar)+'_'+str(trends))
        

In [ ]:
frame_pogr.to_csv('arima_btc_exog_prog.csv')
frame_pogr.to_csv('arima_btc_exog_orig.csv')

frame_pogr.to_csv('arima_snp_exog_orig.csv')
frame_pogr.to_csv('arima_snp_exog_prog.csv')